In [1]:
import os
import pandas as pd
import numpy as np
import re
from shutil import copyfile
import ipdb
import filecmp
import random

In [2]:
cd F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl


[WinError 3] The system cannot find the path specified: 'F:\\Biomedical images\\Train\\NIFTI_MR_512x512\\NIFTI_MR_512x512\\NIFTI_MR_512x512_png_256grey_lvl'
C:\Users\aczd087\Downloads\Master_thesis_infrastructure


In [3]:
imgs_modes=['t1dual','t2spir','inphase','outphase','dicom_anon','ground','source']
src_dir=os.getcwd()
pair_keyword=[['t1dual_inphase','ground'],['t1dual_outphase','ground'],['t2spir','ground']]
dst_grnd_path=''
dst_img_path=''
grnd_sub_repl_tple_dict={'_ground':'_inphase','.png':'_grey_lvl_256.png'}
src_sub_repl_tple_dict={}

In [12]:
file_dict=gen_img_fileset_dict(src_dir,imgs_modes,key_word_sp=1)
file_dict.keys()

F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_outphase
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_outphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_outphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t2spir
F:\Biomedical images\Train\NIFTI_MR_512x512\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t2spir\ground
F:\Biomedica

dict_keys(['t1dual_inphase_ground', 't1dual_inphase_source', 't1dual_outphase_ground', 't1dual_outphase_source', 't2spir_ground', 't2spir_source'])

# Stripping out file information Patient id etc from file name using regex

In [ ]:

final_lst=[]
for file in mask_file_list:
    
    Pat_id_act=int(file[file.lower().find(Pat_str)+len(Pat_str):file.lower().find(img_type_str)])
    
    tmp_img_type=[x for x in img_types_lst if file.lower().find(x)!=-1]
    
    #Getting slice no from file name
    slic_no_lwr_bnd=file.lower().find('slice_no_')
    slic_no_uppr_bnd=file.lower().rfind('_512x512')
    slice_no=int(file[slic_no_lwr_bnd+len('slice_no_'):slic_no_uppr_bnd])
    
    tmp_img=imageio.imread(file)
    
    tmp_img_uniq_val=np.unique(tmp_img,return_counts=True)
    tmp_dict={}
    for int_val,pixel_cnts in list(zip(tmp_img_uniq_val[0],tmp_img_uniq_val[1])):
        int_val_key=mask_intensities[int_val]
        
        tmp_dict[int_val_key]=pixel_cnts
    
    tmp_dict['slice_no']=slice_no
    tmp_dict['image_type']=tmp_img_type[0]
    tmp_dict['Patient_id']=Pat_id_act
    
    final_lst.append(tmp_dict)

In [26]:
cd F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename


# Move data on a per patient basis to different folder set aka test or validation folders

In [28]:
import shutil
dst_path=r'F:\Biomedical images\validation\1_pat_no_32'
pat_no=[32]
pat_no_str=['Pat_'+str(x)+'_' for x in pat_no]
for root,subdir,files in os.walk(os.getcwd()):
    if len(files)>0:
        root_split=root.split('\\')
        
        root_split_rejoin='\\'.join(root_split[-3:])
        tmp_dir=os.path.join(dst_path,root_split_rejoin)
        if not os.path.exists(tmp_dir):
            os.makedirs(tmp_dir)
            
        for file in files:
            
            file_chk=[x for x in pat_no_str if file.find(x)!=-1]
            
            if len(file_chk)==1:
                print(file)
                src_img_path=os.path.join(root,file)
                dst_img_path=os.path.join(tmp_dir,file)
                shutil.move(src_img_path, dst_img_path)
            

Pat_32_t1dual_inphase_slice_no_10_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_11_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_12_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_13_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_14_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_15_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_16_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_17_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_18_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_19_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_1_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_20_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_21_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_22_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_23_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_24_512x512_grey_lvl_128.png
Pat_32_t1dual_inphase_slice_no_25_512x512_grey_lvl_128.pn

Pat_32_t1dual_inphase_slice_no_9_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_10_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_11_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_12_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_13_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_14_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_15_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_16_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_17_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_18_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_19_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_1_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_20_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_21_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_22_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_23_512x512_grey_lvl_256.png
Pat_32_t1dual_outphase_slice_no_24_512x512_

Pat_32_t1dual_inphase_slice_no_14_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_15_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_16_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_17_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_18_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_19_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_1_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_20_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_21_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_22_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_23_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_24_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_25_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_26_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_27_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_28_512x512_grey_lvl_64.png
Pat_32_t1dual_inphase_slice_no_29_512x512_grey_lvl_64.png
Pat_32_t1dual_i

## Rename Ground files to match source file names

In [33]:
cd F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512


In [51]:
grey_lvl_str=['128','256','64']
phase_lvl_str={'inphase':'_inphase','outphase':'_outphase','t2spir':''}
for root,subdir,files in os.walk(os.getcwd()):
    
    if root.find('ground')!=-1:
        #Determine grey level setpoint for analysis 
        grey_lvl_sp=[x for x in grey_lvl_str if root.find(x)!=-1]
        grey_lvl_str_add='_grey_lvl_'+grey_lvl_sp[0]+'.png'
        #Determine ground source level
        phase_lvl_sp=[v for k,v in phase_lvl_str.items() if root.find(k)!=-1]
        
        grnd_sub_repl_tple_dict={'_ground':phase_lvl_sp[0],'.png':grey_lvl_str_add}

        for file in files:
            new_file_name=repl_sub_string(file,grnd_sub_repl_tple_dict)
            old_img=os.path.join(root,file)
            new_img=os.path.join(root,new_file_name)
            #renaming tuple to match srounce image
            os.rename(old_img,new_img)


{'_ground': '_inphase', '.png': '_grey_lvl_128.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\ground
{'_ground': '_outphase', '.png': '_grey_lvl_128.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\ground
{'_ground': '', '.png': '_grey_lvl_128.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\ground
{'_ground': '_inphase', '.png': '_grey_lvl_256.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\ground
{'_ground': '_outphase', '.png': '_grey_lvl_256.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_outphase\ground
{'_ground': '', '.png': '_grey_lvl_256.png'}
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512

In [60]:
#Check to confirm for os path to determine method for analysis 
for root,subdir,files in os.walk(os.getcwd()):
    if len(subdir)==2:        

        dir_lst=[os.path.join(root,x) for x in subdir]
        
        file_lst_tple=list(zip(sorted(os.listdir(dir_lst[0])),
                               sorted(os.listdir(dir_lst[1]))))
                           
        for file_src,file_grnd in file_lst_tple:
            
            if file_src!=file_grnd:
                print(file_src)
                print(file_grnd)
                        

## Function calls for generating file_dictionaries, file tuple sets 

In [12]:
def gen_img_fileset_dict(src_dir,imgs_modes,key_word_sp=1):
    """The purpose of this method is to create a keyword dictoinary of a folders subfolder file directory paths"""
    file_dict={}
    for subdir,root,files in os.walk(src_dir):
        print(subdir)

        if len(files)!=0:
            gt_img_name=[]
            gt_img_name=[''.join(x) for x in imgs_modes if x in subdir.lower()]
            #If two categories 
            if len(gt_img_name)>=key_word_sp:

                tmp_file_lst=[os.path.join(subdir,file) for file in files]
                seperator = '_'
                seperator =seperator.join(gt_img_name)
                file_dict[seperator]=sorted(tmp_file_lst)
    #Sett original name for analysis and munging
    return file_dict

In [13]:
def gen_file_tpl_set_lists(file_dict:dict,pair_keyword: list)->list:
    """The purpose of this method is to generate keyword base file tuple sets"""
    #final list to return list
    rtrn_lst=[]
    #Creating list of keys to iterate through 
    file_sets=list(file_dict.keys()) 
    
    for key in pair_keyword:
        #ipdb.set_trace()
        #Get main image set initially b yimage type i.e. t1dual
        img_type_lst=[x for x in file_sets if x.lower().find(key[0])!=-1]
        #Getting the ground key
        img_grnd_key=[x for x in img_type_lst if x.lower().find(key[1])!=-1]
        #Getting non ground by removing that value from the list 
        img_type_lst.remove(img_grnd_key[0])
        #Generating file list tuple pairs for anaylysis=
        file_pair=list(zip(file_dict[img_type_lst[0]],file_dict[img_grnd_key[0]]))
            
        #Appending final list of tuple for analysis
        rtrn_lst.append(file_pair)
            
    return rtrn_lst


In [14]:
def repl_sub_string(str_val:str,str_rep_dict:dict):
    """The purpose of this method is to itertivley replace substrings in a string with other values"""
    
    # Add escape characters to string keys for regex estimation
    rep = dict((re.escape(k), v) for k, v in str_rep_dict.items()) 
    #Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
    pattern = re.compile("|".join(str_rep_dict.keys()))
    
    return pattern.sub(lambda m: rep[re.escape(m.group(0))],str_val)

In [15]:
def rename_grnd_src_img(file_set_list_tuple:list,
                                     grnd_sub_repl_tple_dict:dict,
                                    src_sub_repl_tple_dict:dict,
                                     dst_grnd_path:str,
                                    dst_img_path:str):
    """The purpose of this method is to detail how """
    
    
    for grnd_nm_path,img_nm_path in file_set_list_tuple:
        
        grnd_nm=os.path.basename(grnd_nm_path)
        img_nm=os.path.basename(img_nm_path)
        
        strp_grnd_nm=repl_sub_string(grnd_nm,grnd_sub_repl_tple_dict)
        strp_img_nm=repl_sub_string(img_nm,src_sub_repl_tple_dict)
        
        dst_grnd_path=os.path.join(dst_grnd_path,strp_grnd_nm)
        dst_img_path=os.path.join(dst_img_path,strp_img_nm)
        
        copyfile(grnd_nm_path, dst_grnd_path)
        copyfile(img_nm_path,dst_img_path)

## Sub sampler by patient

In [29]:
file_list_nm=os.listdir('F:\Biomedical images\Train\MR')
file_list_nm_test=['3','8','39','2','32']
file_list_nm_train=[x for x in file_list_nm if x not in file_list_nm_test]
file_list_nm_train

['1',
 '10',
 '13',
 '15',
 '19',
 '20',
 '21',
 '22',
 '31',
 '33',
 '34',
 '36',
 '37',
 '38',
 '5']

In [30]:
pat_subsampler(r'F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename',
               r'F:\Biomedical images\Train\NIFTI_MR_512x512_250train',
               file_list_nm_train,250)

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\ground
F:\Biomedical imag

In [6]:
def pat_subsampler(data_src_path,data_dst_path,pat_key_word:list,
     No_imgs:int,Pat_weights={},sampling='Uniform'):
    """The purpose of this method is to subsample from each patient an even number of times to ensure no patient specific
    bias is introduced into the process."""
    #Convert patient numbers to stirng 
    str_pat_no_lst=[str(x) for x in pat_key_word if type(x)==str]
    #Confirm file arguments match requiremens
    assert (sampling.lower() in ['uniform','weighted']), "Correct sampling method argument uniform or weighted not inputed"
    #Getting different sampling method if required. 
    pat_dict=subsample_pat_weight(sampling,No_imgs,
                                  str_pat_no_lst,
                                  Pat_weights)
        
    #Iterate through work directory
    for root,subdir,files in os.walk(data_src_path):
        #If two subdirectories found denoted as containing ground and source proceed to second step for analysis 
        sub_dir_chk=[x for x in subdir if x.lower() in ['ground','source']]
        
        if len(sub_dir_chk)==2:
            
            dst_directory_src,dst_directory_grnd=gen_dst_grnd_src_path(root,data_dst_path)
            
            file_dict=gen_img_fileset_dict(root,subdir,key_word_sp=1)
            keys=list(file_dict.keys())
            
            tmp_file_set_sampl=file_dict[keys[0]]
            
            #Getting patient ids
            final_pat_idxs=subsample_patient_idx(tmp_file_set_sampl,pat_dict)
                
            file_grnd_files=[file_dict[keys[0]][i] for i in final_pat_idxs]
            file_img_files=[file_dict[keys[1]][i] for i in final_pat_idxs]
                
            file_tuple_set=list(zip(file_img_files,
                           file_grnd_files))
            #Writing tupe set to file for analysis
            write_subsmpl_file(dst_directory_src,dst_directory_grnd,file_tuple_set)
            

In [7]:
def write_subsmpl_file(dst_directory_src,dst_directory_grnd,file_tuple_set):
    """Write subsampled files to file"""
    for file_img,grnd_img in file_tuple_set:
        
        img_dst_b_name=os.path.basename(file_img)
        grnd_dst_b_name=os.path.basename(grnd_img)
        #Getting destinationnames for copying files across
        img_dst_f_name=os.path.join(dst_directory_src,img_dst_b_name)
        grnd_dst_f_name=os.path.join(dst_directory_grnd,grnd_dst_b_name)
        #Writing subsampled files to new folder set. 
        copyfile(file_img,img_dst_f_name)
        copyfile(grnd_img,grnd_dst_f_name)

In [8]:
def subsample_pat_weight(sampling_arg,
                         total_no_imgs,
                         pat_id_lst,
                         Pat_weights={}):
    """The purpose of this method is to return the number of samples that are to be
    returned per patient class bsaed on weighted input."""
    if sampling_arg.lower()=='uniform':
        img_per_pat=int(round(total_no_imgs/len(pat_id_lst),0))
        pat_dict={k:img_per_pat for k in pat_id_lst}
    else:
        pat_dict={k:int(round(v*total_no_imgs,0)) for k,v in Pat_weights}
        
    return pat_dict

In [23]:
def subsample_patient_idx(tmp_file_set_sampl:list,pat_dict:dict):
    """The purpose is to subsample patient ids indexes from dataset"""
    final_pat_idxs=[]
    #ipdb.set_trace()
    for pat_id,sampl_no in pat_dict.items():
        tmp_pat_str='pat_'+pat_id+'_'
        #Getting indices of specific patient files in list
        pat_indices = [i for i, s in enumerate(tmp_file_set_sampl) if tmp_pat_str in s.lower()]
        #Sampling form indices for analysis
        sampl_pat_ind=random.sample(pat_indices, sampl_no)
        #Add indices to final list
        final_pat_idxs=final_pat_idxs+sampl_pat_ind
    
    return final_pat_idxs

In [10]:
def gen_dst_grnd_src_path(orig_root_dir,dst_path,file_strs={'ground':0,'source':0},dir_depth=-2):
    """The purpose of this method is to generate source and ground destination paths directories
    for the subsampled files"""
    #ipdb.set_trace()
    #Generating directory 
    sub_dir=orig_root_dir.split('\\')[dir_depth:]
    #file_strs
    for img_types,img_paths in file_strs.items():
        
    #Generating source directories for file analysis
        tmp_directory=os.path.join(dst_path,*sub_dir,img_types)

        if not os.path.exists(tmp_directory):
            os.makedirs(tmp_directory)
            
        file_strs[img_types]=tmp_directory
        
    dir_grnd=file_strs['ground']
    dir_img=file_strs['source']
    
    return (dir_img,dir_grnd)
            
        

In [65]:
img_per_pat=20
path_key_word=[1,2,3,4,5,6]

pat_dict={k:img_per_pat for k in path_key_word}
pat_dict

{1: 20, 2: 20, 3: 20, 4: 20, 5: 20, 6: 20}

In [101]:
tr_path=os.path.join('C://trial_path',*root.split('\\')[-2:],'trl')
tr_path

'C://trial_path\\t2spir\\source\\trl'

In [112]:
dst_folder=r'F:\Biomedical images\Train\NIFTI_MR_512x512_250subsample'
trl_root=r'F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase'

gen_dst_grnd_src_path(trl_root,dst_folder)


('F:\\Biomedical images\\Train\\NIFTI_MR_512x512_250subsample\\NIFTI_MR_512x512_png_128grey_lvl\\t1dual_inphase\\source',
 'F:\\Biomedical images\\Train\\NIFTI_MR_512x512_250subsample\\NIFTI_MR_512x512_png_128grey_lvl\\t1dual_inphase\\ground')

In [89]:
trl_lst=[]

for root,subdir,files in os.walk(os.getcwd()):
    
    if len(subdir)==0:
        #print(root.split('\\')[-2:])
        print(root)
        tmp_file_set=[os.path.join(root,file) for file in files]
    
        trl_lst=trl_lst+tmp_file_set
        

F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_inphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t1dual_outphase\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_128grey_lvl\t2spir\source
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\ground
F:\Biomedical images\Train\NIFTI_MR_512x512 seg_rename\NIFTI_MR_512x512\NIFTI_MR_512x512_png_256grey_lvl\t1dual_inphase\sourc